## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print("Number of records in training set " + str(len(train_set)))
print("Number of records in training set " + str(len(test_set)))

Number of records in training set 3718
Number of records in training set 196


In [4]:
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95661

In [5]:
tokens = [pair[0] for pair in train_tagged_words]
V = set(tokens)
len(V)

12115

In [6]:
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [7]:
print("Available tags in the data set\n")
print(T)

Available tags in the data set

{'NOUN', 'CONJ', 'PRT', 'X', 'ADJ', 'PRON', 'DET', 'NUM', 'ADP', '.', 'VERB', 'ADV'}


In [8]:
tagged_words = [tup for sent in nltk_data for tup in sent]
tags = [pair[1] for pair in tagged_words]
from collections import Counter
tag_counts = Counter(tags)
tag_counts

Counter({'NOUN': 28867,
         '.': 11715,
         'NUM': 3546,
         'ADJ': 6397,
         'VERB': 13564,
         'DET': 8725,
         'ADP': 9857,
         'CONJ': 2265,
         'X': 6613,
         'ADV': 3171,
         'PRT': 3219,
         'PRON': 2737})

In [9]:
print(tag_counts.most_common(5))

[('NOUN', 28867), ('VERB', 13564), ('.', 11715), ('ADP', 9857), ('DET', 8725)]


### Emission Probabilities

In [10]:
def word_given_tag(word,tag,train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1] == tag]
    count_tag = len(tag_list)
    word_in_given_tag_list = [pair[0] for pair in tag_list if pair[0] == word]
    count_w_given_tag = len(word_in_given_tag_list)
    return (count_w_given_tag,count_tag)
    

In [11]:
#test the word_given_tag method
print(word_given_tag('will', 'VERB'))
print(word_given_tag('Android', 'NOUN'))
print(word_given_tag('train', 'VERB'))
print(word_given_tag('train', 'NOUN'))

(269, 12867)
(0, 27403)
(2, 12867)
(1, 27403)


In [12]:
print(word_given_tag('opposite', 'NOUN'))

(0, 27403)


In [13]:
# For every tag, test the word_given_tag with a word
for t in T:
    print(t)
    print(word_given_tag('Android', t))

NOUN
(0, 27403)
CONJ
(0, 2155)
PRT
(0, 3045)
X
(0, 6267)
ADJ
(0, 6102)
PRON
(0, 2597)
DET
(0, 8297)
NUM
(0, 3371)
ADP
(0, 9348)
.
(0, 11191)
VERB
(0, 12867)
ADV
(0, 3018)


### Transition Probabilities

In [14]:
def t2_given_t1(t2,t1,train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t == t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1]==t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [15]:
#test t2_given_t1 method
print(t2_given_t1(t2='NOUN', t1='PRT'))

(744, 3045)


In [16]:
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [17]:
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [18]:
tags_df.head()

,NOUN,CONJ,PRT,X,ADJ,PRON,DET,NUM,ADP,.,VERB,ADV
NOUN,0.263000,0.042550,0.043827,0.028719,0.012006,0.004671,0.013101,0.009561,0.176842,0.241287,0.147393,0.017042
CONJ,0.348028,0.000464,0.005104,0.007889,0.116009,0.059397,0.120650,0.041763,0.053364,0.036195,0.156845,0.054292
PRT,0.244335,0.002299,0.001642,0.013136,0.086371,0.018062,0.104433,0.058456,0.021018,0.042365,0.398686,0.009195
X,0.062869,0.010372,0.183022,0.075315,0.017393,0.055210,0.055689,0.002872,0.143769,0.164513,0.203447,0.025531
ADJ,0.699771,0.016716,0.010980,0.021141,0.065552,0.000656,0.004589,0.020649,0.078171,0.065061,0.011963,0.004753


In [19]:
tags_df.loc['.', :]

NOUN    0.222143
CONJ    0.058440
PRT     0.002323
X       0.027611
ADJ     0.044768
PRON    0.065410
DET     0.172460
NUM     0.080958
ADP     0.090430
.       0.094630
VERB    0.088911
ADV     0.051827
Name: ., dtype: float32

### Build the vanilla Viterbi based POS tagger

In [20]:
def vannilla_viterbi(words , train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key,word in enumerate(words):
        p = []
        for tag in T:
            if key ==0:
                transition_p = tags_df.loc['.',tag]
            else:
                transition_p = tags_df.loc[state[-1],tag]
            result = word_given_tag(words[key],tag)
            emission_p = result[0]/result[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [21]:

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print(len(test_tagged_words))

5015


In [22]:
# tagging the test sentences
start = time.time()
tagged_seq = vannilla_viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [23]:
print("Time taken in seconds: ", difference)
print(tagged_seq)

Time taken in seconds:  508.94840693473816
[('The', 'DET'), ('concept', 'NOUN'), ('begot', 'NOUN'), ('a', 'DET'), ('slew', 'NOUN'), ('of', 'ADP'), ('copycats', 'NOUN'), (',', '.'), ('but', 'CONJ'), ('the', 'DET'), ('banks', 'NOUN'), ('stopped', 'VERB'), ('*-1', 'X'), ('promoting', 'VERB'), ('the', 'DET'), ('packages', 'NOUN'), ('.', '.'), ('The', 'DET'), ('Voice', 'NOUN'), ('of', 'ADP'), ('America', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('government', 'NOUN'), ('agency', 'NOUN'), ('that', 'ADP'), ('*T*-1', 'X'), ('broadcasts', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('views', 'NOUN'), ('--', '.'), ('some', 'DET'), ('might', 'VERB'), ('say', 'VERB'), ('propaganda', 'NOUN'), ('--', '.'), ('in', 'ADP'), ('43', 'NUM'), ('languages', 'NOUN'), ('to', 'PRT'), ('130', 'NUM'), ('million', 'NUM'), ('listeners', 'NOUN'), ('around', 'ADP'), ('the', 'DET'), ('world', 'NOUN'), ('.', '.'), ('Several', 'ADJ'), ('Fed', 'NOUN'), ('governors', 'NOUN'), ('in', 'ADP'), ('Washington', 'NOUN'), ('have',

In [24]:
# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)

accuracy

0.9423728813559322

In [25]:
sentences = ['Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.',
             'Android is a mobile operating system developed by Google.',
             'Google and Twitter made a deal in 2015 that gave Google access to Twitter\'s firehose.',
             'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.',
             'Before entering politics, Donald Trump was a domineering businessman and a television personality.',
             'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.',
             'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.',
             'Show me the cheapest round trips from Dallas to Atlanta',
             'I would like to see flights from Denver to Philadelphia.',
             'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.',
             'NASA invited social media users to experience the launch of ICESAT-2 Satellite.']

In [26]:
## Testing
#sentence_test = 'Android is a mobile operating system developed by Google.'
for sentence in sentences:
    words = word_tokenize(sentence)
    start = time.time()
    tagged_seq = vannilla_viterbi(words)
    end = time.time()
    difference = end-start
    print(tagged_seq)
    print(difference)

[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.')]
1.7097039222717285
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]
1.0527949333190918
[('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.')]
1.7371571063995361
[('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', '

In [27]:
print(tagged_seq)
print(difference)

[('NASA', 'NOUN'), ('invited', 'NOUN'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', 'NOUN'), ('Satellite', 'NOUN'), ('.', '.')]
1.310615062713623


### Solve the problem of unknown words

## We will use the combination of lexical and rule tagger

- Numbers are not identified properly
- Unknown if not found in the corpus, will make it NOUN be default

In [48]:
patterns = [(r'.*ing$', 'VERB'),(r'.*ed$', 'VERB'),
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),
    (r'.*', 'NOUN')                  
]
rule_based_tagger = nltk.RegexpTagger(patterns)

lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)

lexicon_tagger.evaluate(test_set)

0.9549351944167498

### Modified version of viterbi algorithm which makes use of combined tagger to determine the probanility for a word with respect to given tag. If word_given_tag method returns the word not present in the corpus, we make use of combined tagger to determine the probability.

In [41]:
def vannilla_viterbi_modified(words , train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key,word in enumerate(words):
        p = []
        for tag in T:
            if key ==0:
                transition_p = tags_df.loc['.',tag]
            else:
                transition_p = tags_df.loc[state[-1],tag]
            result = word_given_tag(words[key],tag)
            derivedProabability = result[0]
            ##Tweaking viterbi algorithm to derive the probability from lexiconRule tagger
            if result[0]==0:
                # It means, given word not present in the corpus
                derivedProabability = rule_based_tagger.evaluate([[(word,tag)]]) 
            emission_p = derivedProabability/result[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [42]:
# tagging the test sentences
start1 = time.time()
tagged_seq1 = vannilla_viterbi_modified(test_tagged_words)
end1 = time.time()
difference1 = end1-start1

print("Time taken in seconds: ", difference1)


Time taken in seconds:  764.2967958450317


#### Evaluating tagging accuracy

In [45]:
check1 = [i for i, j in zip(tagged_seq1, test_run_base) if i == j] 

accuracy1 = len(check1)/len(tagged_seq1)

accuracy1

0.9523429710867398

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [46]:
print("Accuracy with the plain viterbi algorithm: " , str(accuracy))

print("Accuracy with the plain enhanced viterbi algorithm: " , str(accuracy1))

Accuracy with the plain viterbi algorithm:  0.9423728813559322
Accuracy with the plain enhanced viterbi algorithm:  0.9523429710867398


In [47]:
if (accuracy1>accuracy):
    print("Modified version of viterbi algorithm is giving more accuracy than plain vanilla version")

Modified version of viterbi algorithm is giving more accuracy than plain vanilla version


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [34]:

def executeViterbi(sentence):
    words = word_tokenize(sentence)
    start = time.time()
    tagged_seq = vannilla_viterbi_modified(words)
    end = time.time()
    difference = end-start
    print(tagged_seq)
    print(difference)

In [35]:
sentence1 ='Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.'
executeViterbi(sentence1)

[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')]
2.2068560123443604


In [36]:
sentence2 = 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.'
executeViterbi(sentence2)

[('The', 'DET'), ('2018', 'NUM'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NOUN'), ('FIFA', 'NOUN'), ('World', 'NOUN'), ('Cup', 'NOUN'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', 'NOUN'), ('contested', 'VERB'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.')]
3.0790328979492188


In [37]:
sentence3 = 'Google and Twitter made a deal in 2015 that gave Google access to Twitter\'s firehose.'
executeViterbi(sentence3)

[('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'NOUN'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.')]
2.5166239738464355
